In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
print(dataset.size)
print(dataset.index)
print(dataset.values)
print(len(dataset.index))
row, column = dataset.shape
transactions = []
print('Row : ', row, " Column : ", column)
for i in range(0, row):
    # Apiori model is expecting list of columns with string as type
    transactions.append([str(dataset.values[i,j]) for j in range(0,column)])


150020
RangeIndex(start=0, stop=7501, step=1)
[['shrimp' 'almonds' 'avocado' ... 'frozen smoothie' 'spinach'
  'olive oil']
 ['burgers' 'meatballs' 'eggs' ... nan nan nan]
 ['chutney' nan nan ... nan nan nan]
 ...
 ['chicken' nan nan ... nan nan nan]
 ['escalope' 'green tea' nan ... nan nan nan]
 ['eggs' 'frozen smoothie' 'yogurt cake' ... nan nan nan]]
7501
Row :  7501  Column :  20


In [3]:
print(transactions[1:3])
print(len(transactions))

[['burgers', 'meatballs', 'eggs', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['chutney', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']]
7501


In [37]:
#get_ipython().system('pip install apyori')

Defaulting to user installation because normal site-packages is not writeable


# Training Apriori model on the dataset

In [4]:
from apyori import apriori
# min_support= min_support_for_transaction*no.of days/total record --> 3*7/7501= 0.003
# Product should be purchased thrice(3) a day. no of date(7)-Input data collected fr a week. total data - 7501
# min_lift >=3, min_length & max_length=2 since we are planning to offer 1 product as free if they buy 1
# If v offer 1 free product on buying 2 product, then min_length & max_length should be 3.
# if v need to provide 1 product free on buying between 2 to 10 product then min_lift=2 & max_length=11
rules = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2,max_length=2)

#from apyori import apriori
#rules = apriori(transactions = transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2)


# Visualising the result

In [5]:
results = list(rules)
results
# items=frozenset({'light cream', 'chicken'}), support(combination of chicken&light cream bought)=0.0045, confidence=0.29.
# If a customer buys "light cream"(items_base) then he has 29%(confidence) chance of buying "chicken"(items_add)
# Lift - Likelyhood of people who buy other item

[RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

# Putting the result well organised into pandas dataframe

In [11]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

# Displaying the result non sorted

In [12]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


# Displaying the result sorted by descending order

In [16]:
resultsinDataFrame.sort_values('Lift', ascending=False)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710
